In [1]:
import pyTigerGraph as tg

In [2]:
conn = tg.TigerGraphConnection(host="https://mytg.i.tgcloud.io", password="pass123")

In [3]:
print(conn.gsql('''
  CREATE VERTEX ref(PRIMARY_ID id STRING, column1 STRING, column2 STRING, column3 STRING, column4 STRING) WITH primary_id_as_attribute="true"
''', options=[]))

Successfully created vertex types: [ref].


In [27]:
import pandas as pd

In [28]:
df=pd.read_csv('ref.csv', sep=';' , names =['id','column1','column2','column3','column4','column5'])

In [29]:
df

,id,column1,column2,column3,column4,column5
0,1,a1,bbb,ccc,dd,eee
1,2,a2,bbb,ccc,dd,eee
2,3,a3,bbb,ccc,dd,eee
3,4,a1,bbb,ccc,dd,eee
4,5,a2,bbb,ccc,dd,eee
5,6,a3,bbb,ccc,dd,eee
6,7,a1,bbb,ccc,dd,eee
7,8,a2,bbb,ccc,dd,eee
8,9,a3,bbb,ccc,dd,eee
9,10,a1,bbb,ccc,dd,eee


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       12 non-null     object
 1   column1  12 non-null     object
 2   column2  12 non-null     object
 3   column3  12 non-null     object
 4   column4  12 non-null     object
dtypes: object(5)
memory usage: 608.0+ bytes


In [9]:
print(conn.gsql('''CREATE GRAPH Ref(ref)''', options=[]))

The graph Ref is created.


In [10]:
conn.graphname = "Ref"
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)
print(token)

('0p7ecgo5c8b1kk5uisdueo219fj09iis', 1647943031, '2022-03-22 09:57:11')


In [25]:
results=conn.gsql('''
  USE GRAPH Ref
  DROP JOB load_ref
  BEGIN
  CREATE LOADING JOB load_ref FOR GRAPH Ref {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX ref VALUES($0, $1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
''', options=[])
print(results)

Using graph 'Ref'
Successfully dropped jobs on the graph 'Ref': [load_ref].
Successfully created loading jobs: [load_ref].


In [30]:
df.to_csv("ref3.csv", index=None)

In [26]:
import json
#Load the posts file with the 'load_people' job
file='ref2.csv'
results=conn.uploadFile(file, fileTag='MyDataSource', jobName='load_ref')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 13,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "ref",
          "validObject": 13,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]
